HOMEWORK 03

In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression


Dataset

In [28]:
df = pd.read_csv('course_lead_scoring.csv')
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


Data preparation


    Check if the missing values are presented in the features.
    If there are missing values:
        For categorical features, replace them with 'NA'
        For numerical features, replace with with 0.0


In [29]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [30]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [31]:
numerical = ['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']
categorical = ['lead_source', 'industry', 'employment_status', 'location', 'converted']

In [32]:
df[categorical] = df[categorical].fillna('NA')
df[numerical] = df[numerical].fillna(0)
df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

In [33]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NA,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NA,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


Question 1

What is the most frequent observation (mode) for the column industry?

    NA
    technology
    healthcare
    retail  <-- Ans

In [34]:
df['industry'].mode()

0    retail
Name: industry, dtype: object

Question 2

Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features.

What are the two features that have the biggest correlation?

    interaction_count and lead_score
    number_of_courses_viewed and lead_score
    number_of_courses_viewed and interaction_count
    annual_income and interaction_count  <-- Ans

Only consider the pairs above when answering this question.

In [35]:
df[numerical].corr()

,number_of_courses_viewed,annual_income,interaction_count,lead_score
number_of_courses_viewed,1.000000,0.009770,-0.023565,-0.004879
annual_income,0.009770,1.000000,0.027036,0.015610
interaction_count,-0.023565,0.027036,1.000000,0.009888
lead_score,-0.004879,0.015610,0.009888,1.000000


Split the data


    Split your data in train/val/test sets with 60%/20%/20% distribution.
    Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.
    Make sure that the target value converted is not in your dataframe.


In [36]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [37]:
df_full_train = df_full_train.reset_index(drop=True)

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [38]:
len(df_train), len(df_val), len(df_test)

(876, 293, 293)

In [39]:
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

del df_train['converted']
del df_val['converted']
del df_test['converted']

categorical = ['lead_source', 'industry', 'employment_status', 'location']

Question 3


    Calculate the mutual information score between converted and other categorical variables in the dataset. Use the training set only.
    Round the scores to 2 decimals using round(score, 2).

Which of these variables has the biggest mutual information score?

    industry
    location
    lead_source <-- Ans
    employment_status



In [45]:
def mutual_info_converted_score(series):
    return mutual_info_score(series, y_train)

In [46]:
categorical = ['lead_source', 'industry', 'employment_status', 'location']

In [48]:
mi = df_train[categorical].apply(mutual_info_converted_score)
round(mi.sort_values(ascending=False), 2)

lead_source          0.04
employment_status    0.01
industry             0.01
location             0.00
dtype: float64

Question 4


    Now let's train a logistic regression.
    Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
    Fit the model on the training dataset.
        To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
        model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

What accuracy did you get?

    0.64
    0.74
    0.84
    0.94


In [50]:
dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [53]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [64]:
y_pred = model.predict_proba(X_val)[:, 1]
churn_decision = (y_pred >= 0.5)

global_score = (y_val == churn_decision).mean()
global_score

np.float64(0.6996587030716723)

Question 5



    Let's find the least useful feature using the feature elimination technique.
    Train a model using the same features and parameters as in Q4 (without rounding).
    Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
    For each feature, calculate the difference between the original accuracy and the accuracy without the feature.

Which of following feature has the smallest difference?

    'industry' <-- Ans
    'employment_status'
    'lead_score'


In [56]:
def train_model(feature_to_ignore):
    my_df_train = df_train[categorical + numerical]
    del my_df_train[feature_to_ignore]
    X_train = dv.fit_transform(my_df_train.to_dict(orient='records'))

    my_df_val = df_val[categorical + numerical]
    del my_df_val[feature_to_ignore]
    X_val = dv.transform(my_df_val.to_dict(orient='records'))

    model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict_proba(X_val)[:, 1]
    churn_decision = (y_pred >= 0.5)

    return (y_val == churn_decision).mean()

In [60]:
scores = [global_score - train_model(feature) for feature in ['industry', 'employment_status', 'location']]
print(scores)

[np.float64(0.0), np.float64(0.0034129692832763903), np.float64(-0.010238907849829393)]


Question 6


    Now let's train a regularized logistic regression.
    Let's try the following values of the parameter C: [0.01, 0.1, 1, 10, 100].
    Train models using all the features as in Q4.
    Calculate the accuracy on the validation dataset and round it to 3 decimal digits.

Which of these C leads to the best accuracy on the validation set?

    0.01 <-- Ans
    0.1
    1
    10
    100


In [63]:
for c in [0.01, 0.1, 1, 10, 100]:
    model = LogisticRegression(solver='liblinear', C=c, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict_proba(X_val)[:, 1]
    churn_decision = (y_pred >= 0.5)

    score = (y_val == churn_decision).mean()
    print(f'c = {c}, score = {score}')

c = 0.01, score = 0.6996587030716723
c = 0.1, score = 0.6996587030716723
c = 1, score = 0.6996587030716723
c = 10, score = 0.6996587030716723
c = 100, score = 0.6996587030716723
